In [1]:
import ee
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

ee.Authenticate()
projectName = os.getenv("PROJECT")
ee.Initialize(project=projectName)

In [ ]:
lst = ee.ImageCollection('MODIS/061/MOD11A1')

In [3]:
# Initial date of interest (inclusive).
i_date = '2022-01-01'

# Final date of interest (exclusive).
f_date = '2023-01-01'

# Selection of appropriate bands and dates for LST.
band = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [4]:
Hamburg_lon = 9.993682
Hamburg_lat = 53.551086
Hamburg_point = ee.Geometry.Point(Hamburg_lon, Hamburg_lat)

In [5]:
scale = 1000  # scale in meters
lst_hamburg = band.mean().sample(Hamburg_point, scale).first().get('LST_Day_1km').getInfo()
print('Hamburg average daytime LST:', round(lst_hamburg*0.02 -273.15, 2), '°C')
# output is : Hamburg average daytime LST: 16.83 °C

Hamburg average daytime LST: 16.83 °C


In [6]:
lst_Hamburg_full = band.getRegion(Hamburg_point, scale).getInfo()
lst_Hamburg_full[:3] # Preview the output.

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km', 'QC_Day'],
 ['2022_01_01', 9.993757535829678, 53.55306566278528, 1640995200000, None, 2],
 ['2022_01_02', 9.993757535829678, 53.55306566278528, 1641081600000, None, 2]]

In [7]:
import pandas as pd
df = pd.DataFrame(lst_Hamburg_full) #Convert list to dataframe
headers = df.iloc[0]   # Rearrange the header.
df = pd.DataFrame(df.values[1:], columns=headers)   # Rearrange the header.
df = df[['longitude', 'latitude', 'time', "LST_Day_1km" ]].dropna() # Remove rows with null data.
df[ "LST_Day_1km"] = pd.to_numeric(df[ "LST_Day_1km"], errors='coerce')    # Convert to numeric values.
df['datetime'] = pd.to_datetime(df['time'], unit='ms')  # Convert datetime to datetime values.
df = df[['time','datetime',  "LST_Day_1km"   ]] # take interest part
df.head()

,time,datetime,LST_Day_1km
2,1641168000000,2022-01-03,13872
5,1641427200000,2022-01-06,13704
7,1641600000000,2022-01-08,13620
11,1641945600000,2022-01-12,13837
20,1642723200000,2022-01-21,13682
